In [1]:
import numpy as np 
import pandas as pd

In [2]:
train = pd.read_csv('input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x= pd.read_csv('input/sample-data/test_preprocessed.csv')

In [7]:
train_x

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,0,1,0,1,0,0,2015,2,3,24182
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,0,1,1,0,0,2015,5,9,24185
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,0,1,0,1,0,0,2016,2,13,24194
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,0,1,0,0,2015,7,6,24187
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,0,1,1,1,0,2016,9,17,24201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61,1,182.729800,73.393777,1,2000000,189,232,7,17,...,0,0,0,1,1,0,2015,10,21,24190
9996,33,0,167.701136,75.006529,8,9000,426,202,3,19,...,0,0,0,1,1,0,2015,5,28,24185
9997,44,0,145.609998,47.739397,8,1000,370,274,1,11,...,0,0,0,1,0,1,2016,2,29,24194
9998,34,0,165.796017,57.567695,6,5000,291,105,1,13,...,0,1,1,1,1,0,2016,2,27,24194


In [3]:
import xgboost as xgb

In [34]:
class Model:
    def __init__(self, params= None):
        self.model = None
        if params is None:
            self.params={}
        else:
            self.params= params
    def fit(self, tr_x, tr_y, va_x, va_y):
        params= {
            'objective': 'binary:logistic',
            'verbosity': 0,
            'random_state':71
        }
        params.update(self.params)
        num_round= 20 
        dtrain =xgb.DMatrix(tr_x, label = tr_y)
        dvalid =xgb.DMatrix(va_x, label = va_y)
        watchlist= [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals= watchlist)
    
    def predict(self, x):
        data= xgb.DMatrix(x)
        pred= self.model.predict(data)
        return pred
        

In [22]:
#KFold 교차검증 
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
scores=[]
kf = KFold(n_splits=4, shuffle = True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    model = Model()
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred =model.predict(va_x)
    score= log_loss(va_y, va_pred)
    scores.append(score)

print(np.mean(scores))

[0]	train-logloss:0.54088	eval-logloss:0.55003
[1]	train-logloss:0.45269	eval-logloss:0.47182
[2]	train-logloss:0.39482	eval-logloss:0.42026
[3]	train-logloss:0.35197	eval-logloss:0.38520
[4]	train-logloss:0.32021	eval-logloss:0.36150
[5]	train-logloss:0.29673	eval-logloss:0.34463
[6]	train-logloss:0.27610	eval-logloss:0.32900
[7]	train-logloss:0.25886	eval-logloss:0.31670
[8]	train-logloss:0.24363	eval-logloss:0.30775
[9]	train-logloss:0.23153	eval-logloss:0.30092
[0]	train-logloss:0.53892	eval-logloss:0.54864
[1]	train-logloss:0.45219	eval-logloss:0.47148
[2]	train-logloss:0.39574	eval-logloss:0.41997
[3]	train-logloss:0.35476	eval-logloss:0.38413
[4]	train-logloss:0.32218	eval-logloss:0.35626
[5]	train-logloss:0.29945	eval-logloss:0.33910
[6]	train-logloss:0.27783	eval-logloss:0.32552
[7]	train-logloss:0.26326	eval-logloss:0.31573
[8]	train-logloss:0.24780	eval-logloss:0.30592
[9]	train-logloss:0.23369	eval-logloss:0.29596
[0]	train-logloss:0.54332	eval-logloss:0.55058
[1]	train-log

In [25]:
#stratified kfold 
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits= 4, shuffle = True, random_state=71)
for tr_idx, va_idx in kf.split(train_x, train_y): #stratified 는 input을 x, y로 받는다. y= 목적변수
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [26]:
# 4건씩 같은 유저가 있는 데이터였다고 가정한다. 
train_x['user_id']=np.arange(0, len(train_x))//4

In [28]:
# 그룹 k-겹 검증 
from sklearn.model_selection import KFold, GroupKFold

In [30]:
user_id= train_x['user_id']
unique_user_ids= user_id.unique()

In [35]:
scores= []
kf = KFold(n_splits= 4, shuffle= True, random_state =71)
for tr_group_idx, va_group_idx in kf.split(unique_user_ids):
    tr_groups, va_groups= unique_user_ids[tr_group_idx], unique_user_ids[va_group_idx]
    
    #각 샘플의 고객 ID가 train/valid 중 어느쪽에 속해 있느냐에 따라 분할
    is_tr = user_id.isin(tr_groups)
    is_va= user_id.isin(va_groups)
    tr_x,va_x = train_x[is_tr], train_x[is_va]
    tr_y,va_y = train_y[is_tr], train_y[is_va]
    
    model = Model()
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred= model.predict(va_x)
    score= log_loss(va_y, va_pred)
    scores.append(score)
print(np.mean(scores))
# groupkfold는 shuffle 과 난수 생성 기능이 없어 사용하기 어려움, 위의 방식으로 groupfold를 하기를 바람

[0]	train-logloss:0.53873	eval-logloss:0.55094
[1]	train-logloss:0.45085	eval-logloss:0.47461
[2]	train-logloss:0.39357	eval-logloss:0.42569
[3]	train-logloss:0.35167	eval-logloss:0.39392
[4]	train-logloss:0.31953	eval-logloss:0.37083
[5]	train-logloss:0.29667	eval-logloss:0.35357
[6]	train-logloss:0.27383	eval-logloss:0.33850
[7]	train-logloss:0.25814	eval-logloss:0.32850
[8]	train-logloss:0.24245	eval-logloss:0.31838
[9]	train-logloss:0.23020	eval-logloss:0.31210
[10]	train-logloss:0.21996	eval-logloss:0.30484
[11]	train-logloss:0.21182	eval-logloss:0.29906
[12]	train-logloss:0.19945	eval-logloss:0.29243
[13]	train-logloss:0.19100	eval-logloss:0.28571
[14]	train-logloss:0.18473	eval-logloss:0.28208
[15]	train-logloss:0.17579	eval-logloss:0.27887
[16]	train-logloss:0.17118	eval-logloss:0.27575
[17]	train-logloss:0.16555	eval-logloss:0.27345
[18]	train-logloss:0.16147	eval-logloss:0.27067
[19]	train-logloss:0.15699	eval-logloss:0.26815
[0]	train-logloss:0.54119	eval-logloss:0.55071
[1]